In [1]:
import pdfplumber
import re
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
pdf = pdfplumber.open("Magnolia-Warehouse.pdf")
all_pages = pdf.pages
cleaned_lines = []
cleaned_list_of_dicts = []
for each_page in all_pages:
    text_of_each_page = each_page.extract_text()
    all_lines = text_of_each_page.splitlines()
    for each_line in all_lines:
        if each_line == "Address":
            continue
        elif each_line == "Owner Card  Mailing Label  Map and Details Report":
            continue
        elif each_line == "KGIS Parcel Report":
            continue
        else:
            cleaned_lines.append(each_line)
each_dict = {}
for index, cleaned_line in enumerate(cleaned_lines):
    if re.match(r"^09[0-9A-Z]{5,}$", cleaned_line):
        # parcel_id
        each_dict["parcel_id"] = cleaned_line
        each_dict["name"] = cleaned_lines[index + 1]
        cleaned_list_of_dicts.append(each_dict)
        each_dict = {}
    else:
        continue

In [3]:
len(cleaned_list_of_dicts)

192

In [ ]:
# cleaned_list_of_dicts

In [13]:
driver = webdriver.Chrome(ChromeDriverManager().install())

/var/folders/2g/fbs070qj3qd3kw4ntm252trc0000gp/T/ipykernel_92114/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [14]:
def get_all_data(parcel_number, agree):
    driver.get(
        "https://propertyinfo.knoxcountytn.gov/search/commonsearch.aspx?mode=realprop"
    )
    if agree:
        try:
            driver.find_element(By.ID, "btAgree").click()
        except:
            print("No agree modal found")
    driver.find_element(By.ID, "inpParid").send_keys(parcel_number)
    driver.find_element(By.ID, "btSearch").click()
    error_passing = driver.find_elements(By.ID, "searchResults")
    # print(error_passing)
    if len(error_passing) > 0:
        error_passing[0].find_element(By.CLASS_NAME, "SearchResults").click()
    driver.find_element(By.XPATH, '//*[@id="sidemenu"]/ul/li[3]/a').click()
    parcel_id_dirty = driver.find_element(By.CLASS_NAME, "DataletHeaderTop").text
    address_data = driver.find_elements(By.CLASS_NAME, "DataletHeaderBottom")
    parcel_id = parcel_id_dirty.split(": ")[1]
    sales_data = []
    print("address", address_data)
    try:
        sales_details = driver.find_element(By.ID, "Sales Summary")
        # print("about to rows")
        rows = sales_details.find_elements(By.TAG_NAME, "tr")[1:]
        # print("about to row")
        for row in rows:
            all_data = row.find_elements(By.TAG_NAME, "td")
            # print("all_data", all_data)
            if len(all_data) < 5:
                # print("insufficient data")
                continue
            else:
                data = {}
                date = all_data[0].text
                # print("date done")
                book = all_data[1].text
                # print("book done")
                page = all_data[2].text
                # print("page done")
                price = all_data[3].text
                # print("price done")
                grantee = all_data[-1].text
                # print("grantee done")
                address = address_data[-1].text
                # print("address done")

                data["date"] = date
                data["book_number"] = book
                data["page"] = page
                data["sale_price"] = price
                data["grantee_name"] = grantee
                data["parcel_id_new"] = parcel_id
                data["address"] = address
                sales_data.append(data)
    except Exception as e:
        print(e)
        data = {}
        data["date"] = "Not found"
        data["book_number"] = "Not found"
        data["page"] = "Not found"
        data["sale_price"] = "Not found"
        data["grantee_name"] = "Not found"
        data["parcel_id_new"] = "Not found"
        data["address"] = "Not found"
        sales_data.append(data)
    return sales_data

In [6]:
all_sales_data = []

In [15]:
agree = True
for index, each_parcel_dict in enumerate(cleaned_list_of_dicts[151:]):
    sales_data = get_all_data(each_parcel_dict["parcel_id"], agree)
    agree = False
    all_sales_data.extend(sales_data)
#     print(index, "done")

address [<selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="d48451b2-bdf9-4917-97dd-3da93412077e")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="eb301bde-d79b-450f-9241-dd20aa2ffa91")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="45fbb012-7ecd-43ca-bdf6-977c5e837c30")>]
address [<selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="111509b4-2c2e-4c67-94a6-cb1801540742")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="fe602f6e-759b-446d-9119-e75e8c9721b9")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="6adfbed0-cfb0-4bf5-876e-b8a56f0fdc6a")>]
address [<selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="42f73b

address [<selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="08897fe0-4a0f-4950-969b-a9a87ce67c86")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="ab876e7c-fefa-405a-a6f1-40736bf41b2a")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="06471b44-f868-4f0b-a1d4-ed9427c0fec4")>]
address [<selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="2155f3a2-f8fe-4260-9017-65885e8a844b")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="7b5f7efe-9d06-438f-8b9a-49a2132247c4")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="f812bc01-b30f-4219-b335-a82d8e1a9380")>]
address [<selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="2e8a1e

address [<selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="0756617a-df08-49ba-99a3-01b5057c20fe")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="2493d179-2c21-4abf-8b63-e949223750aa")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="63636261-21aa-428f-a598-655f45556f74")>]
address [<selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="16d2d0b5-82d9-48fc-9411-1725fc08d3cf")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="15eead47-86cb-41b4-9a60-11e0654a60bb")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="36aa8614-f06f-49f2-bddf-6af0f9fcdbab")>]
address [<selenium.webdriver.remote.webelement.WebElement (session="c6ea20e50da1744332344cb416c9abb4", element="358ca1

In [16]:
len(all_sales_data)

760

In [ ]:
pd.set_option('display.max_columns', None)
magnolia_df = pd.DataFrame(all_sales_data)

In [ ]:
magnolia_df

In [ ]:
magnolia_df.to_csv('magnolia_NEW.csv')